<a href="https://colab.research.google.com/github/uaamir/bitask/blob/main/WeWorkMotion_UseCase_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:

import sqlite3,os
print(sqlite3.sqlite_version_info)
import matplotlib.pyplot as plt
import pandas as pd


print(sqlite3.sqlite_version_info)
major, minor, _ = sqlite3.sqlite_version_info
if minor < 35:
  !curl https://www.sqlite.org/src/tarball/sqlite.tar.gz?r=release | tar xz
  %cd sqlite/
  !./configure
  !make sqlite3.c
  %cd /content
  !npx degit coleifer/pysqlite3 -f
  !cp sqlite/sqlite3.[ch] .
  !python setup.py build_static build
  !cp build/lib.linux-x86_64-3.7/pysqlite3/_sqlite3.cpython-37m-x86_64-linux-gnu.so \
      /usr/lib/python3.7/lib-dynload/
     
  print("restarting kernel")
  os._exit(00)
else:
  print("sqlite is OK")

(3, 37, 0)
(3, 37, 0)
sqlite is OK


In [3]:
# CREATING THE TABLE

conn = sqlite3.connect('test.db')
print("Opened database successfully");

execute = conn.execute

def dict_factory(cursor, row):
    d = {}
    for idx, col in enumerate(cursor.description):
        d[col[0]] = row[idx]
    return d

conn.row_factory = dict_factory

# def query(sql):
#   cursor = execute(sql)
#   for row in cursor:
#     print(row)   

def query_and_print(sql):
  cursor = execute(sql)
  for row in cursor:
    print(row)

 
def query(sql):
  cursor = execute(sql)
  items = list(cursor)
  return pd.DataFrame.from_dict(items)

Opened database successfully


In [5]:
#Creating the Tables Employee 
query("DROP TABLE employee;")
query("""CREATE TABLE  employee(
id integer PRIMARY Key,
monthly_salary float 
);
""")
query("PRAGMA table_info(employee)")




,cid,name,type,notnull,dflt_value,pk
0,0,id,INTEGER,0,None,1
1,1,monthly_salary,float,0,None,0


In [6]:
#Creating the Tables HR OPs 
#query("DROP TABLE hr_ops;")
query("""CREATE TABLE  hr_ops(
emp_id integer ,
contract_start date,
contract_end date,
country char(2),
uk_work_permit bit,
is_currently_employed bit,
FOREIGN KEY (emp_id) REFERENCES employee(id)
);
""")
query("PRAGMA table_info(hr_ops)")

,cid,name,type,notnull,dflt_value,pk
0,0,emp_id,INTEGER,0,None,0
1,1,contract_start,date,0,None,0
2,2,contract_end,date,0,None,0
3,3,country,char(2),0,None,0
4,4,uk_work_permit,bit,0,None,0
5,5,is_currently_employed,bit,0,None,0


In [7]:
#INSERTING Random Data into Employee Table from CSV file 
#The Data is randomly generated all the fields
df_data = pd.read_csv('https://raw.githubusercontent.com/uaamir/bitask/main/employee.csv', sep=',', )
df_data['monthly_salary'] = df_data['monthly_salary'].astype(float)
df_data.to_sql(name='employee', con=conn, if_exists='append',index=False)
df_data.dtypes

id                  int64
monthly_salary    float64
dtype: object

In [8]:
#INSERTING Random Data into HR_OPS Table from CSV file 
df_data = pd.read_csv('https://raw.githubusercontent.com/uaamir/bitask/main/hr_ops_tbl.csv', sep=',', )
df_data['country'] = df_data['country'].astype(str)
df_data['uk_work_permit'] = df_data['uk_work_permit'].astype(bool)
df_data['is_currently_employed'] = df_data['is_currently_employed'].astype(bool)
df_data['contract_start'] = pd.to_datetime(df_data['contract_start'])
df_data['contract_end'] = pd.to_datetime(df_data['contract_end'])


df_data.to_sql(name='hr_ops', con=conn, if_exists='replace')
df_data.dtypes

emp_id                            int64
contract_start           datetime64[ns]
contract_end             datetime64[ns]
country                          object
uk_work_permit                     bool
is_currently_employed              bool
dtype: object

In [9]:
query("SELECT sqlite_version();")

,sqlite_version()
0,3.37.0


In [10]:
#Task 1 Total Salary Paid in 2020 VS 2019 for all the employees in SPAIN
query("""
SELECT 
  strftime('%Y',hr.contract_end) as Year, 
  COUNT(hr.emp_id) As Number_of_Employees, 
  printf("%,d",SUM(emp.monthly_salary*strftime('%m',hr.contract_end))) as Salary_Paid_Y20,
  (SELECT 
      SUM(emp.monthly_salary) 
    FROM 
      employee emp, hr_ops hr
    WHERE 
      emp.id=hr.emp_id
    AND 
      strftime('%Y',hr.contract_end) = '2019') as Salary_Paid_PY19
FROM 
      hr_ops hr, employee emp
WHERE 
  hr.emp_id=emp.id AND Year='2020' AND hr.country='ES'
GROUP BY 
  Year
LIMIT 10
""")

,Year,Number_of_Employees,Salary_Paid_Y20,Salary_Paid_PY19
0,2020,28,"586,460",1175628.0


In [16]:
#Task 2-1 TOP 10 Customers with Highest Salaries and uk_Work_permit=True and Currently_Employed=True
query("""
SELECT 
  emp.id, 
  emp.monthly_salary
FROM 
  hr_ops hr, employee emp
WHERE 
  hr.emp_id=emp.id AND hr.uk_work_permit=1 AND hr.contract_end>date('now')
ORDER BY emp.monthly_salary DESC

""")

,id,monthly_salary
0,394,4848.0
1,308,4838.0
2,426,4612.0
3,77,4461.0
4,648,3934.0
5,856,3672.0
6,263,3470.0
7,445,3281.0
8,469,3039.0
9,887,2644.0


In [14]:
#TASK 2-2 Spread between the TOP Salaries of UK VS Non UK Permit
query("""
WITH topuk AS (SELECT 
  MAX(emp.monthly_salary) as maxsaluk
 FROM 
  hr_ops hr, employee emp
WHERE 
  hr.emp_id=emp.id AND hr.uk_work_permit=1
), top_non_uk AS (SELECT 
  MAX(emp.monthly_salary) as salnonuk
FROM 
  hr_ops hr, employee emp
WHERE 
  hr.emp_id=emp.id AND hr.uk_work_permit=0
)

select maxsaluk,salnonuk,maxsaluk-salnonuk as diff  from topuk,top_non_uk


""")



,maxsaluk,salnonuk,diff
0,10000.0,4998.0,5002.0
